# Deep Research Agent – Workshop Guide

Dieses Notebook zeigt, wie ein **Deep Research Agent** implementiert wird, der mehrere Recherche-Iterationen durchführt, um umfassende Antworten zu generieren. Der Agent nutzt **Azure AI Search** und **Bing Grounding** für tiefgehende Recherchen.


## Voraussetzungen
- Du hast das Bicep-Skript `bing-grounding-deploy` ausgeführt.
- Du bist per `azd auth login` oder `az login` angemeldet.
- In `tools_and_data/.env` sind alle Werte aus dem Deployment hinterlegt.
- Das Notebook `01_bing_grounding_complete_guide.ipynb` funktioniert einwandfrei.


In [1]:
import os
import json
import time
from pathlib import Path
from dotenv import load_dotenv

ENV_PATH = Path('..') / '..' / '..' / 'tools_and_data' / '.env'
if not ENV_PATH.exists():
    raise FileNotFoundError(f".env nicht gefunden unter {ENV_PATH}")
load_dotenv(ENV_PATH)

# Bing Grounding
bing_resource_name = os.getenv('BING_GROUNDING_RESOURCE_NAME')
bing_endpoint = os.getenv('BING_GROUNDING_ENDPOINT')
bing_api_key = os.getenv('BING_GROUNDING_API_KEY')

# Azure AI Search
search_endpoint = os.getenv('AI_SEARCH_ENDPOINT')
search_index_name = os.getenv('AI_SEARCH_INDEX_NAME', 'docs')

# AI Project (Foundry)
ai_project_account = os.getenv('AI_PROJECT_ACCOUNT_NAME')
ai_project_name = os.getenv('AI_PROJECT_NAME')
project_endpoint = os.getenv('AI_PROJECT_ENDPOINT')
assistant_name = 'deep-research-agent'  # Eigener Agent für Deep Research
assistant_model = os.getenv('AI_PROJECT_AGENT_MODEL', 'gpt-4o')
search_connection_name = os.getenv('AI_PROJECT_SEARCH_CONNECTION_NAME', 'conn-search')
bing_connection_name = os.getenv('AI_PROJECT_BING_CONNECTION_NAME', 'conn-bing-grounding')

subscription_id = os.getenv('AZURE_SUBSCRIPTION_ID')
resource_group = os.getenv('AZURE_RESOURCE_GROUP')

print('Konfiguration:')
print(f"  • Bing Grounding: {'OK' if all([bing_resource_name, bing_endpoint, bing_api_key]) else 'FEHLT'}")
print(f"  • Azure AI Search: {'OK' if search_endpoint else 'FEHLT'}")
print(f"  • AI Project Endpoint: {'OK' if project_endpoint else 'FEHLT'}")
print(f"  • Subscription/Resource Group: {'OK' if subscription_id and resource_group else 'FEHLT'}")
if not all([bing_resource_name, bing_endpoint, bing_api_key, search_endpoint, project_endpoint, subscription_id, resource_group, ai_project_account, ai_project_name]):
    raise EnvironmentError('Bitte alle benötigten Variablen in tools_and_data/.env setzen.')

Konfiguration:
  • Bing Grounding: OK
  • Azure AI Search: OK
  • AI Project Endpoint: OK
  • Subscription/Resource Group: OK


In [2]:
# AI Project (Foundry)
ai_project_account = os.getenv('AI_PROJECT_ACCOUNT_NAME')
ai_project_name = os.getenv('AI_PROJECT_NAME')
project_endpoint = os.getenv('AI_PROJECT_ENDPOINT')
assistant_name = 'deep-research-agent'  # Eigener Agent für Deep Research
assistant_model = os.getenv('AI_PROJECT_AGENT_MODEL', 'gpt-4o-mini')
search_connection_name = os.getenv('AI_PROJECT_SEARCH_CONNECTION_NAME', 'conn-search')
bing_connection_name = os.getenv('AI_PROJECT_BING_CONNECTION_NAME', 'conn-bing-grounding')

subscription_id = os.getenv('AZURE_SUBSCRIPTION_ID')
resource_group = os.getenv('AZURE_RESOURCE_GROUP')
search_index_name = os.getenv('AI_SEARCH_INDEX_NAME', 'docs')

# API Version
API_VERSION = 'v1'

## 1. Umgebung und Variablen laden


## 2. Azure AI Project verbinden


In [3]:
from azure.identity import AzureDeveloperCliCredential, DefaultAzureCredential, ChainedTokenCredential

credential = ChainedTokenCredential(
    AzureDeveloperCliCredential(),
    DefaultAzureCredential(exclude_shared_token_cache_credential=True,
                           exclude_visual_studio_code_credential=True,
                           exclude_visual_studio_credential=True)
)

def build_connection_id(name: str) -> str:
    return (
        f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/"
        f"Microsoft.CognitiveServices/accounts/{ai_project_account}/projects/{ai_project_name}/connections/{name}"
    )

search_connection_id = build_connection_id(search_connection_name)
bing_connection_id = build_connection_id(bing_connection_name)

print('Connection IDs:')
print('  Azure AI Search:', search_connection_id)
print('  Bing Grounding :', bing_connection_id)

Connection IDs:
  Azure AI Search: /subscriptions/888cfad2-25d5-42f6-b278-d237314b6a19/resourceGroups/rg-workshop-it-tage-2025/providers/Microsoft.CognitiveServices/accounts/ai-services-bing-search-it-tage-2025/projects/ai-project-bing-search-it-tage-2025/connections/conn-search
  Bing Grounding : /subscriptions/888cfad2-25d5-42f6-b278-d237314b6a19/resourceGroups/rg-workshop-it-tage-2025/providers/Microsoft.CognitiveServices/accounts/ai-services-bing-search-it-tage-2025/projects/ai-project-bing-search-it-tage-2025/connections/conn-bing-grounding


## 3. Hilfsfunktionen für die Agents-API


In [4]:
import requests

API_VERSION = 'v1'
HEADERS_BASE = {"Content-Type": "application/json"}


def get_token() -> str:
    token = credential.get_token('https://ai.azure.com/.default')
    return token.token


def project_request(method: str, path: str, **kwargs):
    url = f"{project_endpoint}{path}"
    headers = dict(HEADERS_BASE)
    headers.update(kwargs.pop('headers', {}))
    headers['Authorization'] = f"Bearer {get_token()}"
    response = requests.request(method, url, headers=headers, timeout=60, **kwargs)
    if not response.ok:
        raise RuntimeError(f"{method} {path} fehlgeschlagen: {response.status_code} {response.text}")
    if response.text:
        return response.json()
    return None


def list_assistants(name: str):
    after = None
    while True:
        query = f"/assistants?api-version={API_VERSION}"
        if after:
            query += f"&after={after}"
        data = project_request('GET', query)
        for item in data.get('data', []):
            if item.get('name') == name:
                return item
        if not data.get('has_more'):
            break
        after = data.get('last_id')
    return None


def create_assistant():
    payload = {
        "model": assistant_model,
        "name": assistant_name,
        "instructions": (
            "Du bist ein Deep Research Agent. Deine Aufgabe ist es, umfassende Recherchen durchzuführen. "
            "Nutze Azure AI Search für kuratiertes Wissen und Bing Grounding für aktuelles Webwissen. "
            "Analysiere Informationen kritisch, identifiziere Wissenslücken und stelle Folgefragen. "
            "Antworte strukturiert mit klaren Quellenangaben."
        ),
        "tools": [
            {"type": "azure_ai_search"},
            {
                "type": "bing_grounding",
                "bing_grounding": {
                    "search_configurations": [
                        {"connection_id": bing_connection_id}
                    ]
                }
            }
        ],
        "tool_resources": {
            "azure_ai_search": {
                "indexes": [
                    {
                        "index_connection_id": search_connection_id,
                        "index_name": search_index_name,
                        "query_type": "semantic"
                    }
                ]
            }
        }
    }
    return project_request('POST', f"/assistants?api-version={API_VERSION}", json=payload)


def ensure_assistant():
    existing = list_assistants(assistant_name)
    if existing:
        return existing
    print('Deep Research Agent nicht gefunden – wird erstellt ...')
    created = create_assistant()
    print('✅ Deep Research Agent erstellt.')
    return created

### Deep Research Agent prüfen/erstellen


In [5]:
assistant = ensure_assistant()
print('Aktiver Deep Research Agent:')
print(json.dumps({
    'id': assistant['id'],
    'name': assistant['name'],
    'model': assistant['model']
}, indent=2))
assistant_id = assistant['id']

Aktiver Deep Research Agent:
{
  "id": "asst_QEAnoE2NCUIKve3LeDeWNMb5",
  "name": "deep-research-agent",
  "model": "gpt-4o-mini"
}


## 4. Deep Research Funktionen


## 2. Azure AI Project verbinden


In [6]:
from azure.identity import AzureDeveloperCliCredential, DefaultAzureCredential, ChainedTokenCredential

credential = ChainedTokenCredential(
    AzureDeveloperCliCredential(),
    DefaultAzureCredential(exclude_shared_token_cache_credential=True,
                           exclude_visual_studio_code_credential=True,
                           exclude_visual_studio_credential=True)
)

def build_connection_id(name: str) -> str:
    return (
        f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/"
        f"Microsoft.CognitiveServices/accounts/{ai_project_account}/projects/{ai_project_name}/connections/{name}"
    )

search_connection_id = build_connection_id(search_connection_name)
bing_connection_id = build_connection_id(bing_connection_name)

print('Connection IDs:')
print('  Azure AI Search:', search_connection_id)
print('  Bing Grounding :', bing_connection_id)

Connection IDs:
  Azure AI Search: /subscriptions/888cfad2-25d5-42f6-b278-d237314b6a19/resourceGroups/rg-workshop-it-tage-2025/providers/Microsoft.CognitiveServices/accounts/ai-services-bing-search-it-tage-2025/projects/ai-project-bing-search-it-tage-2025/connections/conn-search
  Bing Grounding : /subscriptions/888cfad2-25d5-42f6-b278-d237314b6a19/resourceGroups/rg-workshop-it-tage-2025/providers/Microsoft.CognitiveServices/accounts/ai-services-bing-search-it-tage-2025/projects/ai-project-bing-search-it-tage-2025/connections/conn-bing-grounding


## 3. Hilfsfunktionen für die Agents-API


In [7]:
import requests

API_VERSION = 'v1'
HEADERS_BASE = {"Content-Type": "application/json"}


def get_token() -> str:
    token = credential.get_token('https://ai.azure.com/.default')
    return token.token


def project_request(method: str, path: str, **kwargs):
    url = f"{project_endpoint}{path}"
    headers = dict(HEADERS_BASE)
    headers.update(kwargs.pop('headers', {}))
    headers['Authorization'] = f"Bearer {get_token()}"
    response = requests.request(method, url, headers=headers, timeout=60, **kwargs)
    if not response.ok:
        raise RuntimeError(f"{method} {path} fehlgeschlagen: {response.status_code} {response.text}")
    if response.text:
        return response.json()
    return None


def list_assistants(name: str):
    after = None
    while True:
        query = f"/assistants?api-version={API_VERSION}"
        if after:
            query += f"&after={after}"
        data = project_request('GET', query)
        for item in data.get('data', []):
            if item.get('name') == name:
                return item
        if not data.get('has_more'):
            break
        after = data.get('last_id')
    return None


def create_assistant():
    payload = {
        "model": assistant_model,
        "name": assistant_name,
        "instructions": (
            "Du bist ein Deep Research Agent. Deine Aufgabe ist es, umfassende Recherchen durchzuführen. "
            "Nutze Azure AI Search für kuratiertes Wissen und Bing Grounding für aktuelles Webwissen. "
            "Analysiere Informationen kritisch, identifiziere Wissenslücken und stelle Folgefragen. "
            "Antworte strukturiert mit klaren Quellenangaben."
        ),
        "tools": [
            {"type": "azure_ai_search"},
            {
                "type": "bing_grounding",
                "bing_grounding": {
                    "search_configurations": [
                        {"connection_id": bing_connection_id}
                    ]
                }
            }
        ],
        "tool_resources": {
            "azure_ai_search": {
                "indexes": [
                    {
                        "index_connection_id": search_connection_id,
                        "index_name": search_index_name,
                        "query_type": "semantic"
                    }
                ]
            }
        }
    }
    return project_request('POST', f"/assistants?api-version={API_VERSION}", json=payload)


def ensure_assistant():
    existing = list_assistants(assistant_name)
    if existing:
        return existing
    print('Deep Research Agent nicht gefunden – wird erstellt ...')
    created = create_assistant()
    print('✅ Deep Research Agent erstellt.')
    return created

### Deep Research Agent prüfen/erstellen


In [8]:
assistant = ensure_assistant()
print('Aktiver Deep Research Agent:')
print(json.dumps({
    'id': assistant['id'],
    'name': assistant['name'],
    'model': assistant['model']
}, indent=2))
assistant_id = assistant['id']

Aktiver Deep Research Agent:
{
  "id": "asst_QEAnoE2NCUIKve3LeDeWNMb5",
  "name": "deep-research-agent",
  "model": "gpt-4o-mini"
}


## 4. Deep Research Funktionen


In [9]:
import time
from typing import Tuple, List, Optional


def wait_for_run(thread_id: str, run_id: str, poll: float = 2.0, max_retries: int = 3):
    """Warte auf Abschluss eines Runs mit Retry-Logik"""
    retry_count = 0
    while True:
        try:
            run = project_request('GET', f"/threads/{thread_id}/runs/{run_id}?api-version={API_VERSION}")
            status = run.get('status')
            if status in {'completed', 'failed', 'requires_action', 'cancelled', 'expired'}:
                if status == 'failed':
                    error = run.get('last_error', {})
                    error_code = error.get('code') if isinstance(error, dict) else None
                    
                    # Retry bei Rate Limit
                    if error_code == 'rate_limit_exceeded' and retry_count < max_retries:
                        retry_count += 1
                        wait_time = 30 * retry_count  # Exponentielles Backoff
                        print(f"⚠️  Rate Limit erreicht. Warte {wait_time} Sekunden (Versuch {retry_count}/{max_retries})...")
                        time.sleep(wait_time)
                        # Starte Run neu
                        new_run = project_request(
                            'POST',
                            f"/threads/{thread_id}/runs?api-version={API_VERSION}",
                            json={'assistant_id': assistant_id}
                        )
                        run_id = new_run['id']
                        continue
                    else:
                        print(f"❌ Run fehlgeschlagen: {error}")
                return run
            time.sleep(poll)
        except Exception as e:
            print(f"⚠️  Fehler beim Abrufen des Run-Status: {e}")
            time.sleep(poll)


def extract_message_text(message: dict) -> Tuple[Optional[str], List[dict]]:
    """Extrahiere Text und Quellen aus einer Nachricht"""
    parts = []
    citations = []
    for block in message.get('content', []):
        block_type = block.get('type')
        if block_type == 'output_text':
            output = block.get('output_text', {})
            for item in output.get('content', []):
                text = item.get('text')
                if isinstance(text, dict):
                    value = text.get('value')
                    if value:
                        parts.append(value)
                elif text:
                    parts.append(text)
            for ann in output.get('annotations', []):
                citation = {
                    'title': ann.get('title') or ann.get('content') or ann.get('reference') or 'Quelle',
                    'url': ann.get('url') or ann.get('link')
                }
                citations.append(citation)
        elif block_type == 'text':
            text_obj = block.get('text')
            if isinstance(text_obj, dict):
                value = text_obj.get('value')
                if value:
                    parts.append(value)
            elif text_obj:
                parts.append(text_obj)
    text = '\n'.join(parts).strip() if parts else None
    unique_sources = []
    seen = set()
    for cite in citations:
        key = (cite.get('title'), cite.get('url'))
        if key in seen:
            continue
        seen.add(key)
        if cite.get('title') or cite.get('url'):
            unique_sources.append(cite)
    return text, unique_sources


def get_latest_assistant_message(thread_id: str, attempts: int = 10, delay: float = 1.0) -> Optional[dict]:
    """Hole die neueste Assistant-Nachricht"""
    for _ in range(attempts):
        try:
            messages = project_request('GET', f"/threads/{thread_id}/messages?api-version={API_VERSION}&order=desc&limit=10")
            for message in messages.get('data', []):
                if message.get('role') == 'assistant':
                    return message
        except Exception as e:
            print(f"⚠️  Fehler beim Abrufen der Nachrichten: {e}")
        time.sleep(delay)
    return None


def ask_in_thread(thread_id: str, question: str) -> Tuple[Optional[str], List[dict]]:
    """Stelle eine Frage in einem bestehenden Thread"""
    try:
        project_request(
            'POST',
            f"/threads/{thread_id}/messages?api-version={API_VERSION}",
            json={
                'role': 'user',
                'content': [
                    {'type': 'text', 'text': question}
                ]
            }
        )
        run = project_request(
            'POST',
            f"/threads/{thread_id}/runs?api-version={API_VERSION}",
            json={'assistant_id': assistant_id}
        )
        run_id = run['id']
        finished = wait_for_run(thread_id, run_id)
        if finished.get('status') != 'completed':
            return None, []
        message = get_latest_assistant_message(thread_id)
        if not message:
            return None, []
        answer, sources = extract_message_text(message)
        return answer, sources
    except Exception as e:
        print(f"❌ Fehler bei ask_in_thread: {e}")
        return None, []

## 5. Deep Research Agent Implementierung


In [10]:
class DeepResearchAgent:
    """Deep Research Agent für mehrstufige Recherchen"""
    
    def __init__(self, assistant_id: str, max_iterations: int = 3, delay_between_iterations: float = 5.0):
        self.assistant_id = assistant_id
        self.max_iterations = max_iterations
        self.delay_between_iterations = delay_between_iterations
        self.thread_id = None
        self.all_sources = []
        self.research_steps = []
    
    def start_research(self, initial_question: str):
        """Starte eine neue Recherche"""
        print(f"🔍 Starte Deep Research: {initial_question}")
        print("=" * 80)
        
        # Erstelle neuen Thread
        thread = project_request('POST', f"/threads?api-version={API_VERSION}", json={})
        self.thread_id = thread['id']
        self.all_sources = []
        self.research_steps = []
        
        # Erste Iteration
        answer, sources = ask_in_thread(self.thread_id, initial_question)
        self._add_step(initial_question, answer, sources)
        
        return answer, sources
    
    def deep_dive(self, follow_up_question: str):
        """Führe eine Vertiefung durch"""
        if not self.thread_id:
            raise ValueError("Keine aktive Recherche. Rufe zuerst start_research() auf.")
        
        # Warte zwischen Iterationen, um Rate Limits zu vermeiden
        if self.delay_between_iterations > 0:
            print(f"⏳ Warte {self.delay_between_iterations} Sekunden (Rate Limit Schutz)...")
            time.sleep(self.delay_between_iterations)
        
        print(f"\n🔎 Vertiefung: {follow_up_question}")
        print("-" * 80)
        
        answer, sources = ask_in_thread(self.thread_id, follow_up_question)
        self._add_step(follow_up_question, answer, sources)
        
        return answer, sources
    
    def synthesize(self):
        """Synthetisiere alle Recherche-Ergebnisse"""
        if not self.thread_id:
            raise ValueError("Keine aktive Recherche.")
        
        # Warte vor Synthese
        if self.delay_between_iterations > 0:
            print(f"⏳ Warte {self.delay_between_iterations} Sekunden (Rate Limit Schutz)...")
            time.sleep(self.delay_between_iterations)
        
        print(f"\n📝 Synthetisiere Ergebnisse...")
        print("-" * 80)
        
        synthesis_prompt = (
            "Fasse alle bisherigen Recherche-Ergebnisse zu einer umfassenden, "
            "strukturierten Antwort zusammen. Gliedere die Antwort in Abschnitte "
            "und stelle sicher, dass alle wichtigen Aspekte abgedeckt sind."
        )
        
        answer, sources = ask_in_thread(self.thread_id, synthesis_prompt)
        self._add_step("SYNTHESE", answer, sources)
        
        return answer, sources
    
    def _add_step(self, question: str, answer: str, sources: list):
        """Füge einen Recherche-Schritt hinzu"""
        self.research_steps.append({
            'question': question,
            'answer': answer,
            'sources': sources
        })
        
        # Sammle alle Quellen
        for source in sources:
            key = (source.get('title'), source.get('url'))
            if key not in [(s.get('title'), s.get('url')) for s in self.all_sources]:
                self.all_sources.append(source)
    
    def get_summary(self):
        """Hole eine Zusammenfassung der Recherche"""
        return {
            'total_steps': len(self.research_steps),
            'total_sources': len(self.all_sources),
            'steps': self.research_steps
        }

## 6. Beispiel: Deep Research durchführen


In [11]:
# Erstelle Deep Research Agent mit 10 Sekunden Pause zwischen Iterationen
agent = DeepResearchAgent(assistant_id, max_iterations=3, delay_between_iterations=10.0)

# Starte Recherche
initial_question = "Was ist Neuroplastizität und wie funktioniert sie?"
answer1, sources1 = agent.start_research(initial_question)

print("\n📄 Antwort (Iteration 1):")
print(answer1 or "Keine Antwort erhalten.")
if sources1:
    print(f"\n🔗 Quellen ({len(sources1)}):")
    for src in sources1[:3]:
        print(f"  - {src.get('title')} {src.get('url') or ''}")

🔍 Starte Deep Research: Was ist Neuroplastizität und wie funktioniert sie?

📄 Antwort (Iteration 1):
Neuroplastizität, auch als neuronale Plastizität bezeichnet, ist die Fähigkeit des Gehirns, sich strukturell und funktionell zu verändern als Antwort auf Erfahrungen, Lernen oder Verletzungen. Diese Anpassungsfähigkeit ist entscheidend für den Lernprozess und die Rehabilitation nach neurologischen Schädigungen.

### Funktionsweise der Neuroplastizität

1. **Synaptische Plastizität**: Dies ist die häufigste Form der Neuroplastizität und bezieht sich auf die Stärke der Verbindungen (Synapsen) zwischen Neuronen. Wenn bestimmte neuronale Wege häufiger genutzt werden, können die Synapsen stärker oder schwächer werden (Langzeitpotenzierung oder Langzeitdepression). Diese Veränderungen ermöglichen es uns, neue Fähigkeiten zu erlernen oder Gedächtnisinhalte zu speichern.

2. **Strukturelle Veränderungen**: Durch Lernen oder Wiederholung können Neuronen neue Verbindungen bilden oder bestehende V

### Vertiefung 1: Mechanismen


In [12]:
# Vertiefung: Mechanismen
answer2, sources2 = agent.deep_dive(
    "Welche molekularen und zellulären Mechanismen liegen der Neuroplastizität zugrunde?"
)

print("\n📄 Antwort (Iteration 2):")
print(answer2 or "Keine Antwort erhalten.")
if sources2:
    print(f"\n🔗 Neue Quellen ({len(sources2)}):")
    for src in sources2[:3]:
        print(f"  - {src.get('title')} {src.get('url') or ''}")

⏳ Warte 10.0 Sekunden (Rate Limit Schutz)...

🔎 Vertiefung: Welche molekularen und zellulären Mechanismen liegen der Neuroplastizität zugrunde?
--------------------------------------------------------------------------------

📄 Antwort (Iteration 2):
Die Neuroplastizität beruht auf einer Vielzahl molekularer und zellulärer Mechanismen, die die Anpassungsfähigkeit des Gehirns unterstützen. Diese Mechanismen sind entscheidend für die Bildung von Gedächtnis, Lernen und die Reparatur nach neurologischen Verletzungen. Hier sind einige der wichtigsten Mechanismen:

### 1. **Neurotransmitter-Änderungen**
- **Glutamat**: Der Hauptneurotransmitter für die Erregung im Gehirn. Er spielt eine zentrale Rolle in der Langzeitpotenzierung (LTP), einem Prozess, durch den synaptische Verbindungen gestärkt werden.
- **GABA (Gamma-Aminobuttersäure)**: Der Hauptneurotransmitter für die Hemmung. Veränderungen in den GABAergen Neuronen können die neuronale Erregbarkeit und damit die Plastizität beeinflussen.

### Vertiefung 2: Anwendungen


In [13]:
# Vertiefung: Anwendungen
answer3, sources3 = agent.deep_dive(
    "Welche praktischen Anwendungen hat Neuroplastizität in Medizin und Therapie?"
)

print("\n📄 Antwort (Iteration 3):")
print(answer3 or "Keine Antwort erhalten.")
if sources3:
    print(f"\n🔗 Neue Quellen ({len(sources3)}):")
    for src in sources3[:3]:
        print(f"  - {src.get('title')} {src.get('url') or ''}")

⏳ Warte 10.0 Sekunden (Rate Limit Schutz)...

🔎 Vertiefung: Welche praktischen Anwendungen hat Neuroplastizität in Medizin und Therapie?
--------------------------------------------------------------------------------

📄 Antwort (Iteration 3):
Neuroplastizität hat wichtige praktische Anwendungen in der Medizin und Therapie, insbesondere in der Rehabilitationsmedizin, der Psychiatrie und der Neurologie. Hier sind einige der bedeutendsten Anwendungen:

### 1. **Rehabilitation nach Schlaganfällen und Hirnverletzungen**
- **Physiotherapie und Ergotherapie**: Durch gezielte Übungen, die die Nutzung bestimmter Muskelgruppen und Bewegungsabläufe beinhalten, können Patienten nach einem Schlaganfall die motorischen Fähigkeiten zurückgewinnen. Die Wiederholung dieser Aktivitäten fördert die neuronale Anpassung und Reorganisation.
- **Sprachtherapie**: Nach einem Schlaganfall, der Sprachzentren des Gehirns betrifft, helfen Sprachtherapeuten dabei, neue Wege zur Kommunikation zu erlernen. Die Neur

### Synthese aller Ergebnisse


In [14]:
# Synthetisiere alle Ergebnisse
final_answer, final_sources = agent.synthesize()

print("\n📚 FINALE SYNTHESE:")
print("=" * 80)
print(final_answer or "Keine Synthese erhalten.")

⏳ Warte 10.0 Sekunden (Rate Limit Schutz)...

📝 Synthetisiere Ergebnisse...
--------------------------------------------------------------------------------

📚 FINALE SYNTHESE:
### Übersicht über Neuroplastizität

Neuroplastizität, auch neuronale Plastizität genannt, ist die bemerkenswerte Fähigkeit des Gehirns, sich strukturell und funktionell zu verändern. Diese Anpassungsfähigkeit ist entscheidend für Lernprozesse, Gedächtnisbildung und die Rehabilitation nach neurologischen Verletzungen. Sie umfasst verschiedene Mechanismen, die sowohl auf zellulärer als auch molekularer Ebene wirken.

---

### 1. Definition und Bedeutung

Neuroplastizität beschreibt die Fähigkeit des Gehirns, aufgrund von Erfahrungen, Lernen oder Verletzungen neue neuronale Verbindungen zu bilden oder bestehende zu reorganisieren. Diese Fähigkeit ermöglicht es dem Gehirn, sich an neue Informationen und Veränderungen in der Umwelt anzupassen, was nicht nur für die Entwicklung, sondern auch für die Regeneration nach

## 7. Zusammenfassung der Recherche


In [15]:
summary = agent.get_summary()

print("\n📊 RECHERCHE-ZUSAMMENFASSUNG:")
print("=" * 80)
print(f"Anzahl Iterationen: {summary['total_steps']}")
print(f"Gesammelte Quellen: {summary['total_sources']}")
print()
print("Recherche-Schritte:")
for idx, step in enumerate(summary['steps'], start=1):
    print(f"\n{idx}. {step['question']}")
    print(f"   Quellen: {len(step['sources'])}")
    answer = step['answer'] or 'Keine Antwort erhalten'
    answer_preview = answer[:100] + '...' if len(answer) > 100 else answer
    print(f"   Antwort: {answer_preview}")

print("\n🔗 ALLE QUELLEN:")
print("=" * 80)
for idx, src in enumerate(agent.all_sources, start=1):
    title = src.get('title') or 'Quelle'
    url = src.get('url') or ''
    print(f"{idx}. {title}")
    if url:
        print(f"   {url}")


📊 RECHERCHE-ZUSAMMENFASSUNG:
Anzahl Iterationen: 4
Gesammelte Quellen: 0

Recherche-Schritte:

1. Was ist Neuroplastizität und wie funktioniert sie?
   Quellen: 0
   Antwort: Neuroplastizität, auch als neuronale Plastizität bezeichnet, ist die Fähigkeit des Gehirns, sich str...

2. Welche molekularen und zellulären Mechanismen liegen der Neuroplastizität zugrunde?
   Quellen: 0
   Antwort: Die Neuroplastizität beruht auf einer Vielzahl molekularer und zellulärer Mechanismen, die die Anpas...

3. Welche praktischen Anwendungen hat Neuroplastizität in Medizin und Therapie?
   Quellen: 0
   Antwort: Neuroplastizität hat wichtige praktische Anwendungen in der Medizin und Therapie, insbesondere in de...

4. SYNTHESE
   Quellen: 0
   Antwort: ### Übersicht über Neuroplastizität

Neuroplastizität, auch neuronale Plastizität genannt, ist die b...

🔗 ALLE QUELLEN:


## 8. Nächste Schritte

- Experimentiere mit verschiedenen Themen
- Passe die Anzahl der Iterationen an (`max_iterations`)
- Erweitere den Agent um automatische Folgefragen
- Implementiere eine Bewertung der Quellenqualität
- Exportiere die Ergebnisse in verschiedene Formate (Markdown, PDF, etc.)
